In [1]:
import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt
import keras
import shutil
import re
import tensorflow as tf
from time import time
import pickle 
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50,preprocess_input,decode_predictions
from keras.utils import load_img, img_to_array
from keras.models import Model,load_model
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding,LSTM,Conv2D
from keras.layers import Add 
from keras_preprocessing.sequence import pad_sequences

In [2]:
%%cmd
pip install wandb
pip install keras-unet

Microsoft Windows [Version 10.0.22621.608]
(c) Microsoft Corporation. All rights reserved.

(machineLearning) d:\Python Project\Photo Coloring>pip install wandb

(machineLearning) d:\Python Project\Photo Coloring>pip install keras-unet

(machineLearning) d:\Python Project\Photo Coloring>

In [3]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
sess

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



In [4]:
trainImageColorful = list(map(lambda x : "data/train/"+x ,os.listdir("data/train/")))
testImageColorful = list(map(lambda x : "data/test/"+x ,os.listdir("data/test/")))
valImageColorful = list(map(lambda x : "data/val/"+x ,os.listdir("data/val/")))


In [5]:
len(trainImageColorful)

6000

In [6]:
from keras_unet.models import custom_unet

customUNetModel = custom_unet(
    input_shape=(224, 224, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid')

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [7]:
def preprocess_image_to_gray(img):
    img = cv2.resize(cv2.cvtColor(cv2.imread(img),cv2.COLOR_BGR2GRAY),(224,224))
    img = img/255
    img = np.repeat(img[..., np.newaxis], 3, -1)
   
    img = np.expand_dims(img,axis=0)
   
    return img

def preprocess_image(img):
    img = cv2.resize(cv2.imread(img),(224,224))
    img = img/255
    return img



In [8]:
trainImageColorful[0]

'data/train/1001773457_577c3a7d70.jpg'

In [9]:
start = time()

valX={}
valY={}


for ix,img in enumerate(valImageColorful):
    

   
    valX[img.split("/")[2][:-4]] = preprocess_image_to_gray(img)
    valY[img.split("/")[2][:-4]] = preprocess_image(img)
 

    if ix%100==0:
        print("Encoding val image - "+str(ix))

print("Time taken in seconds = ", time()-start)

start = time()

testX={}
testY={}


for ix,img in enumerate(testImageColorful):
    

   
    testX[img.split("/")[2][:-4]] = preprocess_image_to_gray(img)
    testY[img.split("/")[2][:-4]] = preprocess_image(img)
 

    if ix%100==0:
        print("Encoding test image - "+str(ix))

print("Time taken in seconds = ", time()-start)

Encoding val image - 0
Encoding val image - 100
Encoding val image - 200
Encoding val image - 300
Encoding val image - 400
Encoding val image - 500
Encoding val image - 600
Encoding val image - 700
Encoding val image - 800
Encoding val image - 900
Encoding val image - 1000
Time taken in seconds =  33.86139702796936
Encoding test image - 0
Encoding test image - 100
Encoding test image - 200
Encoding test image - 300
Encoding test image - 400
Encoding test image - 500
Encoding test image - 600
Encoding test image - 700
Encoding test image - 800
Encoding test image - 900
Time taken in seconds =  18.473795652389526


In [10]:
import pickle as pkl


with open("pickles/process_val_imagesX.pkl","wb") as process_pickle:
    pickle.dump(valX,process_pickle)
with open("pickles/process_val_imagesY.pkl","wb") as process_pickle:
    pickle.dump(valY,process_pickle)



with open("pickles/process_test_imagesX.pkl","wb") as process_pickle:
    pickle.dump(testX,process_pickle)
with open("pickles/process_test_imagesY.pkl","wb") as process_pickle:
    pickle.dump(testY,process_pickle)

""" 
with open("pickles/process_test_imagesX.pkl","rb") as f:
    testX = pkl.load(f)
with open("pickles/process_test_imagesY.pkl","rb") as f:
    testY = pkl.load(f)
with open("pickles/process_val_imagesX.pkl","rb") as f:
    valX = pkl.load(f)
with open("pickles/process_val_imagesY.pkl","rb") as f:
    valY = pkl.load(f) """

' \nwith open("pickles/process_test_imagesX.pkl","rb") as f:\n    testX = pkl.load(f)\nwith open("pickles/process_test_imagesY.pkl","rb") as f:\n    testY = pkl.load(f)\nwith open("pickles/process_val_imagesX.pkl","rb") as f:\n    valX = pkl.load(f)\nwith open("pickles/process_val_imagesY.pkl","rb") as f:\n    valY = pkl.load(f) '

In [11]:
start = time()

trainX={}
trainY={}


for ix, img in enumerate(trainImageColorful):
    
    img=trainImageColorful[ix]
   
    
    trainX[img.split("/")[2][:-4]] = preprocess_image_to_gray(img)
    trainY[img.split("/")[2][:-4]] = preprocess_image(img)
   
 

    if ix%100==0:
        print("Encoding image - "+str(ix))

print("Time taken in seconds = ", time()-start)

Encoding image - 0
Encoding image - 100
Encoding image - 200
Encoding image - 300
Encoding image - 400
Encoding image - 500
Encoding image - 600
Encoding image - 700
Encoding image - 800
Encoding image - 900
Encoding image - 1000
Encoding image - 1100
Encoding image - 1200
Encoding image - 1300
Encoding image - 1400
Encoding image - 1500
Encoding image - 1600
Encoding image - 1700
Encoding image - 1800
Encoding image - 1900
Encoding image - 2000
Encoding image - 2100
Encoding image - 2200
Encoding image - 2300
Encoding image - 2400
Encoding image - 2500
Encoding image - 2600
Encoding image - 2700
Encoding image - 2800
Encoding image - 2900
Encoding image - 3000
Encoding image - 3100
Encoding image - 3200
Encoding image - 3300
Encoding image - 3400
Encoding image - 3500
Encoding image - 3600
Encoding image - 3700
Encoding image - 3800
Encoding image - 3900
Encoding image - 4000
Encoding image - 4100
Encoding image - 4200
Encoding image - 4300
Encoding image - 4400
Encoding image - 4500


In [12]:
with open("pickles/process_train_imagesX.pkl","wb") as process_pickle:
    pickle.dump(trainX,process_pickle)
with open("pickles/process_train_imagesY.pkl","wb") as process_pickle:
    pickle.dump(trainY,process_pickle)

In [13]:
import pickle as pkl
with open("pickles/process_train_imagesX.pkl","rb") as f:
    trainX = pkl.load(f)

with open("pickles/process_train_imagesY.pkl","rb") as f:
    trainY = pkl.load(f)

In [14]:
trainImageColorful

['data/train/1001773457_577c3a7d70.jpg',
 'data/train/1002674143_1b742ab4b8.jpg',
 'data/train/1007129816_e794419615.jpg',
 'data/train/1007320043_627395c3d8.jpg',
 'data/train/1009434119_febe49276a.jpg',
 'data/train/1012212859_01547e3f17.jpg',
 'data/train/1015584366_dfcec3c85a.jpg',
 'data/train/101654506_8eb26cfb60.jpg',
 'data/train/101669240_b2d3e7f17b.jpg',
 'data/train/1016887272_03199f49c4.jpg',
 'data/train/1019077836_6fc9b15408.jpg',
 'data/train/1019604187_d087bf9a5f.jpg',
 'data/train/1020651753_06077ec457.jpg',
 'data/train/1022454332_6af2c1449a.jpg',
 'data/train/1022975728_75515238d8.jpg',
 'data/train/1024138940_f1fefbdce1.jpg',
 'data/train/1026685415_0431cbf574.jpg',
 'data/train/1028205764_7e8df9a2ea.jpg',
 'data/train/1030985833_b0902ea560.jpg',
 'data/train/103106960_e8a41d64f8.jpg',
 'data/train/103205630_682ca7285b.jpg',
 'data/train/1032460886_4a598ed535.jpg',
 'data/train/1034276567_49bb87c51c.jpg',
 'data/train/104136873_5b5d41be75.jpg',
 'data/train/10425903

In [15]:
def data_generator(trainX,num_photos_per_batch):
    
    X, y = [], []

    n = 0

    while True:

        for key in trainImageColorful:
            n +=1

            photo = trainX[key.split("/")[2][:-4]]

            photoColor = trainY[key.split("/")[2][:-4]]
        
            print(photo.shape)
            print(photoColor.shape)
                
            X.append(photo)
                   
            y.append(photoColor)

            if n==num_photos_per_batch:
                u = np.array(X)
            
                w = np.array(y)
                yield([u],w)
                X,  y = [], []
                n = 0

In [16]:
epochs = 10
number_pics_per_bath = 10 
steps = len(trainImageColorful)//number_pics_per_bath

In [17]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [18]:
import wandb 
wandb.init()

In [ ]:
customUNetModel.compile("sgd", "binary_crossentropy", ["acc"])

In [ ]:
for i in range(epochs):
    generator = data_generator(trainX==trainX,num_photos_per_batch=number_pics_per_bath)
 
    customUNetModel.fit(generator,epochs=10,steps_per_epoch=steps,verbose=1,callbacks=[wandb.keras.WandbCallback()])
    customUNetModel.save("./model_weights/model_"+str(i)+".h5")

(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
(2048,)
(224, 224, 3)
Epoch 1/10


OperatorNotAllowedInGraphError: in user code:

    File "c:\Users\hosma\anaconda3\envs\machineLearning\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hosma\AppData\Local\Temp\ipykernel_22416\3943823561.py", line 2, in custom_loss  *
        print(list(y_true))

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.
